<a href="https://colab.research.google.com/github/PabloMendieta03/UFV-VisualizacionDeDatos/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version

Python 3.10.12


# Ejercicio de histórico de ventas: Planificación

## Carga de la Base de Datos

In [3]:
# Importación de librerías
import pandas as pd
from google.colab import drive
import plotly.express as px

In [4]:
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# Crear la columna de fecha
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# mostrar los datos
display(data)

Mounted at /content/drive


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,DATE
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05,2023-12-01
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05,2023-12-01
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04,2023-12-01
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05,2024-12-01
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05,2023-09-01
...,...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06,2024-02-01
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06,2024-07-01
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00,2024-01-01
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05,2024-03-01


## Análisis de los Datos

### **Variable 'SCENARIO'**



In [5]:
# Obtener las categorías únicas en 'SCENARIO'
categorias = data['SCENARIO'].unique()
print("Categorías en 'SCENARIO':", categorias)

Categorías en 'SCENARIO': ['AI_forecast' 'actual']


¿ Número de acutals y AI_forecast ?

In [6]:
# Contar los registros de escenario
ActualsForecast = data['SCENARIO'].value_counts()

# Calcular porcentajes
total = ActualsForecast.sum()
porcentajes = (ActualsForecast / total) * 100

# Gráfico de Barras
fig = px.bar(porcentajes,
             x=porcentajes.values,
             y=porcentajes.index,
             orientation='h', # Barras en horizontal
             labels={'y': 'Escenario', 'x': 'Porcentaje (%)'},
             title='Distribución de Actual vs AI_forecast en Porcentajes',
             text=porcentajes.values  # Valores en las barras
)

# Truncar los porcentajes y poner %
fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')

# Mostrar el gráfico
fig.show()



### **Varible 'COUNTRY'**

¿ Número de Países ?

In [7]:
# Obtener el número de países únicos
paises = data['COUNTRY'].nunique()
print(f"El número de países es: {paises}")

El número de países es: 9


In [8]:
# Contar los registros de COUNTRY
VacesPais = data['COUNTRY'].value_counts()

# Calcular porcentajes
totalPais = VacesPais.sum()
porcentajesPais = (VacesPais / totalPais) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesPais.index, # Etiquetas
    values=porcentajesPais.values, # Valores númericos
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3, # Pie Chart tipo Donut
    color_discrete_sequence=px.colors.sequential.Reds_r # Secuencias de colores
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent')

# Mostrar el gráfico
fig.show()

### **Variable 'SUBBRAND'**

¿ Número de Productos ?

In [9]:
# Obtener el número de productos únicos
productos = data['SUBBRAND'].nunique()
print(f"El número de productos diferentes es: {productos}")

El número de productos diferentes es: 6


In [10]:
# Contar los registros de SUBBRAND
VacesProducto = data['SUBBRAND'].value_counts()

# Calcular porcentajes
totalProducto = VacesProducto.sum()
porcentajesProducto = (VacesProducto / totalProducto) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesProducto.index,
    values=porcentajesProducto.values,
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent')

fig.show()

### **Variable 'FORECAST'**

In [11]:
# Obtener el número de forecasts distintos
forecastsDistintos = data['FORECAST'].nunique()
print(f"El número de forecasts distintos es: {forecastsDistintos}")

El número de forecasts distintos es: 12


### **Histórico de Datos: Actuals/Forecast**

In [12]:
# Agrupar por fecha y escenario, sumando el amount
ActualForecastData = data.groupby(['DATE', 'SCENARIO'], as_index=False)['AMOUNT'].sum()

# Graficar
fig = px.line(ActualForecastData, x='DATE', y='AMOUNT',
              color='SCENARIO',
              title='Histórico de Datos: Actuals vs Forecast',
              labels={'AMOUNT': 'Cantidad', 'DATE': 'Fecha'})


fig.show()

### **Horizontes de Previsión:**

In [13]:
# Contar las frecuencias de tamaños
frecuencias = data.groupby(['COUNTRY', 'SUBBRAND', 'FORECAST']).size().value_counts().reset_index()

# Renombrar las columnas
frecuencias.columns = ['Tamaño', 'Frecuencia']

# Gráfico de Pastel
fig = px.pie(
    frecuencias,
    names='Tamaño',
    values='Frecuencia',
    title='Tamaños de Horizontes de Previsión',
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent+label')
fig.show()

Podemos observar que el 61.2 % de los horizontes de previsión tienen un valor de 18.

## 1. Distribución de las ventas Realizadas:



### a) Por Países:

In [14]:
# Agrupar por país y escenario, sumando las ventas
ventasPais = data.groupby(['COUNTRY', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Calcular las ventas totales por país
ventasPaisTotales = ventasPais.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Ordenar los países de mayor a menor según las ventas
ventasPaisTotales = ventasPaisTotales.sort_values(by='AMOUNT', ascending= False)

# Unir las ventas por país y escenario con las ventas totales
ventasPais['COUNTRY'] = pd.Categorical(ventasPais['COUNTRY'], categories=ventasPaisTotales['COUNTRY'], ordered=True)
ventasPais = ventasPais.sort_values('COUNTRY')

# Gráfico de Barras
fig = px.bar(ventasPais,
             x='COUNTRY',
             y='AMOUNT',
             color='SCENARIO',
             title='Distribución de Ventas por País y Escenario',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Ventas Totales'},
             text_auto=True
)

# Modo de Gráfico de Barras Agrupado
fig.update_layout(barmode='group')

# Hacemos que el texto se vea fuera de las barras
fig.for_each_trace(lambda t: t.update(textposition='outside'))

fig.show()

### b) Por mes y año:

In [15]:
# Agrupamos por mes y año y sumamos las ventas
ventas_por_mes = data.groupby(data['DATE'].dt.to_period('M'))['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Líneas
fig = px.line(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año (Totales)',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

In [16]:
# Agrupamos por país, mes y año y sumamos las ventas
ventas_por_mes = data.groupby(['COUNTRY', data['DATE'].dt.to_period('M')])['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Área
fig = px.area(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año por País',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              color='COUNTRY'
)

fig.show()

### c) Por marca:

In [17]:
# Agrupamos por submarca y sumamos las ventas
ventas_por_submarca = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Gráfico de Pastel
fig = px.pie(ventas_por_submarca, values='AMOUNT', names='SUBBRAND',
             title='Distribución de Ventas por Marca',
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Ventas Totales'},
             hole = 0.3,
             color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()

## 2. Tendencia y Estacionalidad:

### a) Todas las ventas del país con menos ventas

País con menos ventas:

In [18]:
# Encontramos el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]
print(paisMenosVentas)

COUNTRY             Spain
SCENARIO           actual
AMOUNT      8131265.97752
Name: 17, dtype: object


In [19]:
# Filtrar datos para el país con menos ventas y escenario 'actual'
datosSpainActual = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'actual')]

# Sumar el número de ventas
ventasSpainActual = datosSpainActual.groupby(['DATE', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Gráfico de Líneas
fig = px.line(ventasSpainActual, x='DATE', y='AMOUNT', color='SCENARIO',
              title=f'Ventas en {paisMenosVentas["COUNTRY"]}',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

* **TENDENCIA:**

No se observa tendencia en las ventas de España, Las ventas en el mismo mes de los dos años son similares, por lo que ni aumentan ni disminuyen de 2023 a 2024.

* **ESTACIONALIDAD:**

Sí podemos oberservar una estacionalidad anual.

### b) Marca más vendida:

In [20]:
datosPaisMenosVentas = data[data['COUNTRY'] == paisMenosVentas['COUNTRY']]

# Agrupar por 'SUBBRAND' y calcular el total de ventas para cada una
ventasPorSubmarca = datosPaisMenosVentas.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()


fig = px.pie( ventasPorSubmarca,
              names='SUBBRAND',
              values='AMOUNT',
              title=f'Distribución de Ventas Totales por Submarca en {paisMenosVentas["COUNTRY"]}',
              labels={'SUBBRAND': 'Submarca', 'AMOUNT': 'Ventas Totales'},
              hole=0.3,
              color_discrete_sequence=px.colors.qualitative.Pastel2
)

fig.show()

# Marca más vendida
submarcaMasVendida = ventasPorSubmarca.loc[ventasPorSubmarca['AMOUNT'].idxmax()]
print(submarcaMasVendida)

SUBBRAND     Pepsi Max (L3)
AMOUNT      85251053.001145
Name: 3, dtype: object


In [21]:
# Filtrar los datos para la marca más vendida en el país con menos ventas y el escenario 'actual'
datosMarcaMasVendidaActual = datosPaisMenosVentas[
    (datosPaisMenosVentas['SUBBRAND'] == submarcaMasVendida['SUBBRAND']) &
    (datosPaisMenosVentas['SCENARIO'] == 'actual')
]

# Crear la columna 'DATE' para organizar los datos por mes
datosMarcaMasVendidaActual['DATE'] = pd.to_datetime(datosMarcaMasVendidaActual[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar por mes y sumar las ventas
ventasPorMesActual = datosMarcaMasVendidaActual.groupby('DATE')['AMOUNT'].sum().reset_index()


fig = px.line(
    ventasPorMesActual,
    x='DATE',
    y='AMOUNT',
    title=f'Tendencia de Ventas Mensuales de {submarcaMasVendida["SUBBRAND"]}',
    labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
    markers=True
)

# Mostrar el gráfico
fig.show()

<ipython-input-21-fef8360aaa8f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



* **TENDENCIA:**

No se observa tendencia en las ventas de Pepsi Max, Las ventas en el mismo mes de los dos años son similares, por lo que ni aumentan ni disminuyen de 2023 a 2024.

* **ESTACIONALIDAD:**

Sí podemos oberservar una estacionalidad anual.

### COSAS MAL

In [22]:

# Filtrar datos para el país con menos ventas y escenario 'AI_forecast'
datosPaisForecast = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'AI_forecast')]

# Sumar el número de ventas
ventasFechaForecast = datosPaisForecast.groupby(['DATE', 'SCENARIO'])['AMOUNT'].sum().reset_index()

Podemos observar que los valores de predicción son mucho mayores que los valores reales. Por lo tanto, vamos a estudiar por qué pasa esto.





In [23]:
# Filtrar los datos para el país Spain, el escenario AI_forecast, y una submarca específica
datosSpainForecast = data[(data['COUNTRY'] == 'Spain') &
                          (data['SCENARIO'] == 'AI_forecast') &
                          (data['FORECAST'] == 'AI_P10F') &
                          (data['SUBBRAND'] == 'Pepsi Regular (L3)')]

# Ordenar por fecha
datosSpainAI = datosSpainForecast.sort_values(by='DATE')

# Mostrar la tabla
print(len(datosSpainAI))
datosSpainAI.head()

54


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,DATE
6798,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,153166.298991,2023-10-01
13733,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,144399.950702,2023-10-01
2062,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,142858.058146,2023-10-01
15074,Spain,Pepsi Regular (L3),2023,11,AI_forecast,AI_P10F,2023.0,127347.454712,2023-11-01
6572,Spain,Pepsi Regular (L3),2023,11,AI_forecast,AI_P10F,2023.0,141309.803551,2023-11-01


Obersavamos que hay **54 datos**, para **18 meses**, por lo que 3 cantidades diferentes para cada mes. Vamos a hacer la media de los AMOUNTS.

In [24]:
# Agrupar por fecha y calcular la media de los valores de 'AMOUNT' para cada mes
mediaSpainForecast = datosSpainForecast.groupby('DATE')['AMOUNT'].mean().reset_index()
mediaSpainForecast['SCENARIO'] = 'AI_forecast'  # Añadir columna de escenario

# Mostrar la tabla de medias por mes
print(len(mediaSpainForecast))

18


In [25]:
# Filtrar datos para el país con menos ventas y escenario 'actual'
datosPaisActual = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'actual')]
ventasFechaActual = datosPaisActual.groupby('DATE')['AMOUNT'].sum().reset_index()
ventasFechaActual['SCENARIO'] = 'actual'

# Filtrar datos para el país con menos ventas y escenario 'AI_forecast'
datosPaisForecast = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'AI_forecast')]


# Concatenar ambos conjuntos de datos
ventasFecha = pd.concat([mediaSpainForecast, ventasFechaActual])

# Gráfico de Líneas combinadas
fig = px.line(ventasFecha, x='DATE', y='AMOUNT', color='SCENARIO',
              title=f'Tendencia de Ventas en {paisMenosVentas["COUNTRY"]}: Actual y AI_forecast',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
              )

fig.show()